Optomised Model A SHAP

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import shap
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

In [3]:
model_path = r"C:\Users\Jarushen\Desktop\Masters Thesis\Code\Models\Final_A"

# Load the model
model = tf.keras.models.load_model(model_path)



ValueError: Unable to restore custom object of type _tf_keras_metric. Please make sure that any custom layers are included in the `custom_objects` arg when calling `load_model()` and make sure that all layers implement `get_config` and `from_config`.

In [ ]:


# Function to load and preprocess the image
def load_preprocess_image(img_path, target_size):
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale the image
    return img_array


# Directory of the class you want to analyze
class_dir = r"C:\Users\Jarushen\Desktop\Masters Thesis\Images\Fynbos_Dataset_B\Leucadendron salignum"  # Replace with your class directory

# Directory to save the SHAP plots
save_dir = r"C:\Users\Jarushen\Desktop\Masters Thesis\Report_Final\images\Results\OM_A"  # Replace with your desired save directory
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Initialize SHAP DeepExplainer
explainer = shap.DeepExplainer(model, np.zeros((1, 150, 150, 3)))

# Iterate through each image in the specified class directory
for img_file in os.listdir(class_dir):
    if 'IMG' in img_file:
        img_path = os.path.join(class_dir, img_file)
        preprocessed_image = load_preprocess_image(img_path, (150, 150))

        # Predict the class of the image
        predictions = model.predict(preprocessed_image)
        predicted_class = np.argmax(predictions, axis=1)[0]

        # Compute SHAP values for the predicted class
        shap.explainers._deep.deep_tf.op_handlers["AddV2"] = shap.explainers._deep.deep_tf.passthrough 
        shap.explainers._deep.deep_tf.op_handlers["FusedBatchNormV3"] = shap.explainers._deep.deep_tf.passthrough
        
        shap_values = explainer.shap_values(preprocessed_image)
        shap_values_for_predicted_class = shap_values[predicted_class][0]

        # Plot SHAP values
        plt.figure()
        shap.image_plot(shap_values_for_predicted_class, preprocessed_image[0],show=False)

        # Save the plot in the specified directory
        plt.savefig(os.path.join(save_dir, f'shap_output_{img_file}_leucadendron_salignum.png'))
        plt.close()  # Close the plot to free memory

print("All SHAP analysis plots saved.")



In [4]:
import tensorflow as tf
print(tf.__version__)

2.10.0
